In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#predictions from Lag-LLama model
pred_df = pd.read_pickle('/content/drive/My Drive/lag_llama_elec_BE_C1w_P24h_pred_interval.pkl')
pred_df = pred_df[['time', 'actual', 'step1']]
pred_df.rename(columns = {'step1':'pred'}, inplace = True)

In [ ]:
pred_df

,time,actual,pred
0,2016-10-29 00:00:00,53.029999,"[39.29847, 46.85613, 41.73501, 65.316124, 31.1..."
1,2016-10-29 01:00:00,47.680000,"[43.50239, 34.276146, 48.267384, 36.765198, 29..."
2,2016-10-29 02:00:00,46.000000,"[40.094704, 25.225666, 48.30184, 41.265545, 32..."
3,2016-10-29 03:00:00,42.799999,"[22.819115, 52.48851, 29.54736, 41.47263, 12.2..."
4,2016-10-29 04:00:00,39.430000,"[51.675724, 46.054085, 27.322285, 39.187393, 4..."
...,...,...,...
1507,2016-12-30 19:00:00,57.419998,"[68.23987, 0.0, 74.03635, 67.45534, 79.6763, 7..."
1508,2016-12-30 20:00:00,57.560001,"[65.06552, 55.568752, 60.818283, 53.051685, 64..."
1509,2016-12-30 21:00:00,52.180000,"[48.274376, 50.48092, 45.180943, 54.457733, 54..."
1510,2016-12-30 22:00:00,54.380001,"[52.927383, 50.26232, 44.08007, 50.18791, 49.9..."


In [ ]:
def bootstrap_mean(data, size):
  """Calculates the mean of a bootstrap sample."""
  bootstrap_sample = np.random.choice(data, size, replace=True)
  return np.mean(bootstrap_sample)

# Create 20 bootstrap mean columns
for i in range(100):
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))


# Calculate the mean of the 'pred' column
pred_df['point_pred'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 1))

bootstrap_mean_cols = [col for col in pred_df.columns if 'bootstrap_mean_' in col]
pred_df['bootstrap_mean_all'] = pred_df[bootstrap_mean_cols].mean(axis=1)

<ipython-input-7-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))
<ipython-input-7-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pred_df[f'bootstrap_mean_{i+1}'] = pred_df['pred'].apply(lambda x: bootstrap_mean(x, 40))
<ipython-input-7-70d06fc0dc97>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [ ]:
# prompt: calculate MSE for pred_mean and pred_bootstrap_all with respect to actual

from sklearn.metrics import mean_squared_error

# Calculate MSE for pred_mean
mse_mean = mean_squared_error(pred_df['actual'], pred_df['point_pred'])
print(f"MSE for point_pred: {mse_mean}")

# Calculate MSE for pred_bootstrap_all
mse_bootstrap_all = mean_squared_error(pred_df['actual'], pred_df['bootstrap_mean_all'])
print(f"MSE for pred_bootstrap_all: {mse_bootstrap_all}")


MSE for point_pred: 1650.900146484375
MSE for pred_bootstrap_all: 1297.090087890625
